Desafio Protheus


1.   Instala o pacote FPDF no ambiente ***Python***






In [ ]:
!pip install fpdf

2.   Importa as bibliotecas necessárias para o projeto


In [ ]:
import sqlite3
import requests
import json
from datetime import datetime
from fpdf import FPDF

3.   Cria uma conexão com o banco de dados ***desafio_protheus.bd*** e cria as tabelas:


In [ ]:
# Cria uma conexão com o banco de dados desafio_protheus.bd
conn = sqlite3.connect('desafio_protheus.bd')

# Cria a tabela Acao no BD
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Acao (
        idAcao INTEGER PRIMARY KEY,
        simbolo TEXT,
        nome TEXT
    )
''')

# Cria a tabela Cotacao no BD
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Cotacao (
        idAcao INTEGER,
        idCotacao INTEGER PRIMARY KEY,
        Cotacao FLOAT,
        ValorMercado FLOAT,
        VolumeTransacoes FLOAT,
        Moeda TEXT,
        Data DATETIME,
        FOREIGN KEY (idAcao) REFERENCES Acao(idAcao)
    )
''')



4.   Faz o download da lista de ações disponíveis na ***API***




In [ ]:
# URL da API
url = 'https://brapi.dev/api/available'

# Faz a solicitação GET
response = requests.get(url)

# Verifica se a resposta foi bem-sucedida
if response.status_code != 200:
    print('Erro ao conectar na API')
    exit()

# Converte a resposta para um objeto Python
data = json.loads(response.text)

# Cria um objeto Python com as siglas
siglas = {'siglas': data['stocks']}

# Escreve o objeto Python em um arquivo JSON
with open('siglas.json', 'w') as f:
    json.dump(siglas, f)
print('---------------------------------')
print('|Siglas armazenadas com sucesso!|')
print('---------------------------------')



5.   Faz uma solicitação GET para a API da ***brapi.dev*** para cada sigla presente no arquivo ***siglas.json***




In [ ]:
# Função para inserir os dados da cotação no banco de dados
def inserir_cotacao_bd(simbolo, nome, cotacao, valor_mercado, volume_transacoes, moeda, data):
    conn = sqlite3.connect('desafio_protheus.bd')
    cursor = conn.cursor()
    
    # Verificar se já existe uma entrada com o mesmo idAcao e Data
    cursor.execute("SELECT idCotacao FROM Cotacao JOIN Acao ON Cotacao.idAcao = Acao.idAcao WHERE Acao.simbolo = ? AND Cotacao.Data = ?", (simbolo, data))
    resultado = cursor.fetchone()
    
    if resultado:
        # Se já existe, atualizar os valores
        id_cotacao = resultado[0]
        cursor.execute("UPDATE Cotacao SET Cotacao=?, ValorMercado=?, VolumeTransacoes=?, Moeda=? WHERE idCotacao=?", (cotacao, valor_mercado, volume_transacoes, moeda, id_cotacao))
        print(f'{simbolo} atualizado com sucesso!')
    else:
        # Se não existe, inserir uma nova linha
        cursor.execute("INSERT INTO Acao (simbolo, nome) VALUES (?, ?)", (simbolo, nome))
        id_acao = cursor.lastrowid
        cursor.execute("INSERT INTO Cotacao (idAcao, Cotacao, ValorMercado, VolumeTransacoes, Moeda, Data) VALUES (?, ?, ?, ?, ?, ?)",
                        (id_acao, cotacao, valor_mercado, volume_transacoes, moeda, data))
        print(f'{simbolo} inserido com sucesso!')
    
    conn.commit()
    conn.close()

# URL da API
url_base = 'https://brapi.dev/api/'

total_siglas = len(siglas)
contador = 0

# Faz a solicitação GET para cada sigla no dicionário "siglas"
with open('siglas.json') as f:
    siglas = json.load(f)['siglas']
    for sigla in siglas:
        contador += 1
        url = url_base + 'quote/' + sigla
        response = requests.get(url)
        if response.status_code == 200:
            data = json.loads(response.text)['results'][0]
            simbolo = data.get('symbol')
            nome = data.get('shortName')
            cotacao = data.get('regularMarketPrice')
            valor_mercado = data.get('marketCap')
            volume_transacoes = data.get('regularMarketVolume')
            moeda = data.get('currency')
            data = datetime.fromisoformat(data['regularMarketTime'][:-1]).strftime('%Y-%m-%d %H:%M:%S')
            inserir_cotacao_bd(simbolo, nome, cotacao, valor_mercado, volume_transacoes, moeda, data)
            print(f'{simbolo} inserido com sucesso!')
        else:
            print(f'Erro ao consultar {sigla}')




6.   Gera o PDF




In [ ]:
# cria um objeto PDF
pdf = FPDF()

# adiciona uma página ao PDF
pdf.add_page()

# define o tamanho e a fonte do texto do cabeçalho
pdf.set_font('Arial', 'B', 20)

# adiciona o cabeçalho com o título
pdf.cell(0, 10, 'Desafio Programador Protheus Unoesc', 0, 1, 'C')
pdf.ln()

# define o tamanho e a fonte do texto
pdf.set_font('Arial', 'B', 16)

# abre uma conexão com o banco de dados
conn = sqlite3.connect('desafio_protheus.bd')

# busca os dados no banco de dados
cursor = conn.execute("SELECT simbolo, nome, Cotacao, ValorMercado, VolumeTransacoes, Moeda, Data FROM Acao INNER JOIN Cotacao ON Acao.idAcao = Cotacao.idAcao")

# percorre os resultados da consulta e adiciona ao PDF
for row in cursor:
    pdf.cell(40, 10, 'Símbolo: ' + row[0])
    pdf.ln()
    pdf.cell(40, 10, 'Nome: ' + str(row[1]))
    pdf.ln()
    pdf.cell(40, 10, 'Cotação: ' + str(row[2]))
    pdf.ln()
    pdf.cell(40, 10, 'Valor de Mercado: ' + str(row[3]))
    pdf.ln()
    pdf.cell(40, 10, 'Volume de Transações: ' + str(row[4]))
    pdf.ln()
    pdf.cell(40, 10, 'Moeda: ' + row[5])
    pdf.ln()
    pdf.cell(40, 10, 'Data: ' + row[6])
    pdf.ln()
    pdf.ln()

# fecha a conexão com o banco de dados
conn.close()

# salva o PDF
pdf.output('dados.pdf', 'F')
